# LlamaIndex - Private Setup

Using GPT4ALL and our HuggingFace embeddings, we will injest [Chapter 3 of the recent IPCC Climate Report](https://www.ipcc.ch/report/ar6/wg2/chapter/chapter-3/), which covers oceans and coastal ecosystems. Using llama-index, this PDF is injested and vectorized, and questions can be answered about anything from this 172 paged report.

Climate reports are long and tedious to read, so this demo will also help give some insight to the latest findings from the IPCC!

Inspired by the recent popularity of [PrivateGPT](https://github.com/imartinez/privateGPT), this notebook will walk you through a llama-index setup that uses entirely local models. In this notebook, we use GPT4ALL and huggingface embeddings, which should run decently well on CPU alone. If you had more resources, we also provide some links further down for setting up any LLM from huggingface and running on GPU.

## Dependencies Setup

### Setup Line Wrapping

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

### Download gpt4all model

In [1]:
!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

--2023-05-15 17:52:45--  https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin
Resolving gpt4all.io (gpt4all.io)... 172.67.71.169, 104.26.1.159, 104.26.0.159, ...
Connecting to gpt4all.io (gpt4all.io)|172.67.71.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785248281 (3.5G)
Saving to: ‘ggml-gpt4all-j-v1.3-groovy.bin’

ggml-gpt4all-j-v1.3 100%[===================>]   3.52G  60.2MB/s    in 71s     

2023-05-15 17:53:57 (50.7 MB/s) - ‘ggml-gpt4all-j-v1.3-groovy.bin’ saved [3785248281/3785248281]



### Download 2023 IPPC Climate Report - Chapter 3 on Oceans and Coastal Ecosystems (172 Pages)

In [2]:
!wget https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf

--2023-05-15 17:55:18--  https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf
Resolving www.ipcc.ch (www.ipcc.ch)... 104.20.23.161, 172.67.16.59, 104.20.24.161, ...
Connecting to www.ipcc.ch (www.ipcc.ch)|104.20.23.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21752444 (21M) [application/pdf]
Saving to: ‘IPCC_AR6_WGII_Chapter03.pdf’

IPCC_AR6_WGII_Chapt 100%[===================>]  20.74M  5.51MB/s    in 3.8s    

2023-05-15 17:55:22 (5.41 MB/s) - ‘IPCC_AR6_WGII_Chapter03.pdf’ saved [21752444/21752444]



### Download extra packages

In [3]:
!pip install pymupdf pygpt4all llama-index sentence_transformers accelerate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 6.2 MB/s eta 0:00:00a 0:00:01
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.3/273.3 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.0/246.0 kB 51.8 MB/s eta 0:00:00
  Using cached torch-1.12.1-cp310-cp310-manylinux1_x86_64.whl (776.3 MB)
  Created wheel for pygpt4all: filename=pygpt4all-1.1.0-py3-none-any.whl size=5846 sha256=092dc734f00228b782af52653b3207c5927778da73e1451690d5837285148af1
  Stored in directory: /home/Startupcolors/.cache/pip/wheels/7f/28/5c/336b6a4fb972a1ce86beee6222db2f01cfc33fd5b9046ba434
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=967a581995c45c43a1dd57e8820eb1dd91323ae09b0f85fa3593b49cd4512290
  Stored in directory: /home/Startupcolors/.cache/pip/wheels/0a/f5/dd/9d00836c

## Documents setup

Here, we use PyMuPDFReader to quickly load all 172 pages of the climate report PDF. The `metadata=True` option will automatically set some helpful information like page numbers and filename, to help us keep track of sources.

In [4]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index import (
    GPTVectorStoreIndex, 
    LangchainEmbedding, 
    LLMPredictor, 
    ServiceContext, 
    StorageContext, 
    download_loader,
    PromptHelper
)

/home/Startupcolors/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
PyMuPDFReader = download_loader("PyMuPDFReader")

In [6]:
documents = PyMuPDFReader().load(file_path='./IPCC_AR6_WGII_Chapter03.pdf', metadata=True)

# ensure document texts are not bytes objects
for doc in documents:
    doc.text = doc.text.decode()

In [7]:
# print a document to test. Each document is a single page from the pdf, with appropriate metadata
documents[10]

Document(text='3\n389\nOceans and Coastal Ecosystems and Their Services  \nChapter 3\noverlapping climate-induced drivers and non-climate drivers confound \nimplementation and assessment of the success of marine adaptation, \nrevealing the complexity of attempting to maintain marine ecosystems \nand services through adaptation. SROCC assessed with high confidence \nthat while the benefits of many locally implemented adaptations exceed \ntheir disadvantages, others are marginally effective and have large \ndisadvantages, and overall, adaptation has a limited ability to reduce the \nprobable risks from climate change, being at best a temporary solution \n(Bindoff et\xa0al., 2019a). SROCC also concluded that a portfolio of many \ndifferent types of adaptation actions, effective and inclusive governance, \nand mitigation must be combined for successful adaptation (Bindoff \net\xa0al., 2019a). The portfolio of adaptation measures has now been defined \n(Section\xa03.6.2), and individual and

## CPU Llama Index
The GPT4ALL setup follows the instructions from [langchain](https://python.langchain.com/en/latest/modules/models/llms/integrations/gpt4all.html).

Then, the model is wrapped in the LLMPredictor class from llama-index. 

Keep in mind this current setup will run on CPU. If you have access to a GPU, you could also run any LLM from huggingface for improved speed and performance. More details available on huggingface LLMs and example notebooks [here](https://gpt-index.readthedocs.io/en/latest/how_to/customization/custom_llms.html#example-using-a-huggingface-llm).

Lastly, the embeddings are downloaded and run locally using huggingface. These will automatically run on GPU if you have CUDA installed, otherwise they will run on CPU.

In [8]:
local_llm_path = './ggml-gpt4all-j-v1.3-groovy.bin'
llm = GPT4All(model=local_llm_path, backend='gptj', streaming=True, n_ctx=1024)
llm_predictor = LLMPredictor(llm=llm)

gptj_model_load: loading model from './ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 4505.45 MB
gptj_model_load: memory_size =   896.00 MB, n_mem = 57344
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [9]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

Note that due to model limitations (and considerations for speed), these settings are non-standard, but should help ensure reasonable resource usage with GPT4ALL

In [10]:
prompt_helper = PromptHelper(max_input_size=1024, num_output=256, max_chunk_overlap=-1000)
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embed_model,
    prompt_helper=prompt_helper,
    node_parser=SimpleNodeParser(text_splitter=TokenTextSplitter(chunk_size=512, chunk_overlap=50))
)

In [11]:
# Quick sanity test that the LLM works (seems to only work if you run this here!)
print(llm._call("hello world"))


Hello! How can I assist you today?


### Create the Index

This step will break each document into nodes, and create an embedding vector for each node using our `embed_model`. This may take a several minutes if running on CPU (this is a large climate report)!

In [12]:
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

In [13]:
index.storage_context.persist(persist_dir="./storage")

#### (Optional) Load the Index if already saved

In [14]:
from llama_index import load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context, service_context=service_context)

### Try Asking a question

Due to processing constraints, setting `similarity_top_k=1` is an ideal setting. Otherwise, responses will be quite slow due to the speed of CPU inference. Either way, expect to wait a minute or two before the response is streaming.

In [15]:
query_engine = index.as_query_engine(streaming=True, similarity_top_k=1, service_context=service_context)

In [1]:
response_stream = query_engine.query("What are the main climate risks to our Oceans?")
response_stream.print_response_stream()

NameError: name 'query_engine' is not defined

The above response is hallucinated slightly, since all that was not availabe in the provided top-1 releveant node. In any case, it grounded the response and gave a generally OK response

## GPU Llama Index

As stated earlier, if you have a modest GPU available (at least 15GB of VRAM), you can speed things up considerably.

This next section will setup a new predictor from Huggingface, using the Writer/camel-5b-hf model (which is also conviently licensed for commericial use).

(If you are running in colab, switch to a GPU instance first!)

### LLM + Embed Model Setup

In [ ]:
# setup prompts - specific to Camel
from llama_index.prompts.prompts import SimpleInputPrompt

# This will wrap the default prompts that are internal to llama-index
# taken from https://huggingface.co/Writer/camel-5b-hf
query_wrapper_prompt = SimpleInputPrompt(
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

In [ ]:
import torch
from llama_index.llm_predictor import HuggingFaceLLMPredictor

# NOTE: the first run of this will download/cache the weights, ~20GB
hf_predictor = HuggingFaceLLMPredictor(
    max_input_size=2048, 
    max_new_tokens=256,
    temperature=0.25,
    do_sample=False,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="Writer/camel-5b-hf",
    model_name="Writer/camel-5b-hf",
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    model_kwargs={"torch_dtype": torch.bfloat16}
)

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

service_context = ServiceContext.from_defaults(chunk_size_limit=512, llm_predictor=hf_predictor, embed_model=embed_model)

### Construct index using GPU

Since we are using a GPU now, the emebddings are much faster to generate! Plus, now we are working with larger chunks, which enables the embeddings to represent the text and enable better retrieval.

In [ ]:
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
index.storage_context.persist(persist_dir="./storage")

#### (Optional) Load if already saved

In [ ]:
from llama_index import load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context, service_context=service_context)

### Query using GPU

With a GPU, the response will begin streaming very quickly. Camel is an excellent model given it's modest size of 5B parameters!

In [ ]:
query_engine = index.as_query_engine(streaming=True, similarity_top_k=3, service_context=service_context)

In [ ]:
response_stream = query_engine.query("What are the main climate risks to our Oceans?")
response_stream.print_response_stream()

Compared to GPT4ALL, this response is more complete and accurate. You can inspect the source nodes below:

In [ ]:
print(response_stream.source_nodes)